## Imports & Data read

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')


In [ ]:
path_data_full = r"data/data_merged_20250922.parquet"
no_na_path = r"data/data_no_na.pkl"
df = pd.read_pickle(no_na_path, )
df.shape 
df

### Data preprocess


In [ ]:
cols_explicatives = [
    "annee",
    "codecommune",
    "pvotepreviouspvoteD",
    "pvotepvoteD",


    "popcommunes/pop",
    "popcommunesvbbm/vbbm",
    "agesexcommunes/prop014",
    "agesexcommunes/prop60p",
    "agesexcommunes/perage",
    "diplomescommunes/pbac",
    "diplomescommunes/psup",
    "diplomescommunes/nodip",
    "cspcommunes/pouvr",
    "cspcommunes/pcadr",
    "cspcommunes/pchom",
    "revcommunes/revratio",
]
# X = df[cols_explicatives]
# y_par = df["pvoteppar"]
# y_G = df["pvotepvoteG"]
# y_D = df["pvotepvoteD"]

In [ ]:
data = pd.read_parquet(path_data_full, columns=cols_explicatives)
np.unique(data["codecommune"])

#### Traitement X

In [ ]:
# Toutes les colonnes comptent des valeurs manquantes.
# Un objectif dans un premier plan serait d'étudier des techniques ML/Statistiques permettant de combler ces données manquantes.
# Pour l'instant nous étudierons la sous partie du dataset sans valeurs manquantes. 
X.isna().sum()

In [ ]:
# Nous n'avons plus accès à 90% du dataset.
# Ce n'est pas envisageable de produire des résulats avec cette sous-partie, pour l'instant utilisons là pour analyser la structure du dataset. 

X_no_na = X.dropna()
X_no_na.shape

In [ ]:
X_no_na = pd.read_pickle(r"data/data_no_na.pkl")

#### Traitement y (label)

In [ ]:
def data_process(y: pd.Series, X: pd.DataFrame) -> pd.Series:
    mask = ~(y.isna() | X.isna().any(axis=1))
    return y[mask]

y_par_no_na = data_process(y_par, X)
y_G_no_na = data_process(y_G, X)
y_D_no_na = data_process(y_D, X)


#### Sampling

In [ ]:
size_sample = int(1e4)
index_sample = X_no_na.sample(size_sample, random_state=42).index

X_sample = X_no_na.loc[index_sample]

y_par_sample = y_par_no_na.loc[index_sample]
y_G_sample = y_G_no_na.loc[index_sample]
y_D_sample = y_D_no_na.loc[index_sample]


In [ ]:
## Etude de la distribution de "annee" dans notre mini-dataset
## On voit que seules les élections présidentielle récente n'ont pas de données manquantes parmi les colonnes selectionnées.

X_sample["annee"].value_counts()

### Etude du poids des features dans la prédiction y pour 2022 et 2017 séparemment.

In [ ]:
def train_test_model_rf(X, y, n_estimators=600, min_samples_leaf=2, max_depth=6, min_samples_split=4):
    X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.15,
    random_state=42
    )

    rf = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        min_samples_split=min_samples_split,
        n_jobs=-1,
        bootstrap=True,
        random_state=42
    )

    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)

    ### Métriques classiques d'évaluation du modèle
    print("R2:", r2_score(y_test, y_pred))
    print("RMSE:", mean_squared_error(y_test, y_pred) ** 0.5)
    print(f"Std de y: {np.std(y_test):.3f} --- Moyenne de y: {np.mean(y_test):.3f}")

    ### Calcul de l'importance par permutation des features
    perm = permutation_importance(
    rf, X_test, y_test,
    n_repeats=15,
    random_state=42,
    n_jobs=-1
    )

    pi = pd.DataFrame({
        'feature': X.columns,
        'importance_mean': perm.importances_mean,
        'importance_std': perm.importances_std
    }).sort_values('importance_mean', ascending=False)

    print("Permutation Importance: ", pi)
    return rf, pi



index_2017 = X_sample[X_sample["annee"]==2017].index 
index_2022 = X_sample[X_sample["annee"]==2022].index 

X_sample_2022 = X_sample.loc[index_2022]
X_sample_2017 = X_sample.loc[index_2017]


y_D_sample_2017 = y_D_sample.loc[index_2017]
y_D_sample_2022 = y_D_sample.loc[index_2022]


### ---------- Model fit & test -----------------

cols_to_drop = ['pvotepreviouspvoteD', 'annee', 'pvotepvoteD']
# Pour l'instant on s'attardera sur les votes droites.
print("Résultats 2022: \n")
rf_22, pi_22 = train_test_model_rf(X_sample_2022.drop(cols_to_drop, axis=1), y_D_sample_2022)
print("\n---------------------------------------------------------\n")
print("Résultats 2017: \n")
rf_17, pi_17 = train_test_model_rf(X_sample_2017.drop(cols_to_drop, axis=1), y_D_sample_2017)


In [ ]:
pi_17.sort_values(by="importance_mean", ascending=False)

In [ ]:
pi_22.sort_values(by="importance_mean", ascending=False)


**Commentaires Permutation Importance**

Grâce à la "permutation imortance", nous répondons à la question suivante pour chaque feature: "À quel point est-ce que le modèle se dégrade lorsque nous méleangeons au hasard les donneés de la feature?"
Nous arrivons ainsi à mieux déceler l'impact de chaque feature. 



**Commentaire premier modèle**

Nous avons pour les deux cas un RMSE autour de 0.1 avec y qui se situe etre 0 et 1 avec un écart-type de 0.12. 
Ainsi, nous n'expliquons qu'à peine 10% de la variance de y avec notre modèle, un R2 bas confirme cet hypothèse. Cela implique que 15 features sur une année peinent à expliquer les résultats d'élections, ce qui est prévisible. Il y'a beaucoup plus de facteurs qui entre en jeu et nous n'avons pas utiliser les caractéristiques temporelles de notre dataset. 

### Entraînement du même modèle sur 2017 et une partie de 2022 pour prédire 2022.


In [ ]:
# Only 770 code commune incommon 
X_sample_2022.shape, X_sample_2017.shape, X_sample_2022.merge(X_sample_2017, on="codecommune", how="inner").shape

In [ ]:
cols_for_delta = [ 
    "popcommunes/pop",
    "popcommunesvbbm/vbbm",
    "agesexcommunes/prop014",
    "agesexcommunes/prop60p",
    "agesexcommunes/perage",
    "diplomescommunes/pbac",
    "diplomescommunes/psup",
    "diplomescommunes/nodip",
    "cspcommunes/pouvr",
    "cspcommunes/pcadr",
    "cspcommunes/pchom",
    "revcommunes/revratio",
    "rsacommunes/perrsa",
    "capitalimmobiliercommunes/prixm2ratio",
    "naticommunes/pimmigre",
]

In [ ]:

codes_communes_inters = np.intersect1d(
    X_sample_2022["codecommune"].values,
    X_sample_2017["codecommune"].values,
)

X_2022_tmp = X_sample_2022.copy()

# 3. Aligner 2017 et 2022 par codecommune
X22 = X_2022_tmp.set_index("codecommune").loc[codes_communes_inters]
X17 = X_sample_2017.set_index("codecommune").loc[codes_communes_inters]

new_df = {}
for col in cols_for_delta:
    name = f"delta_{col}"
    new_df[name] = X22[col] - X17[col]

delta_df = pd.DataFrame(new_df)

X22.reset_index(drop=True, inplace=True)         
delta_df.reset_index(drop=True, inplace=True)

X_augmented_2022 = pd.concat([X22, delta_df], axis=1)

# Oblique rf / Partial dependence plot

In [ ]:
X_augmented_2022.dropna(inplace=True)

In [ ]:
rf_chrono, pi_chrono = train_test_model_rf(X_augmented_2022.drop(["pvotepvoteD", "annee"], axis=1), 
                                           X_augmented_2022["pvotepvoteD"],
                                           n_estimators=300,
                                           min_samples_leaf=4,
                                           min_samples_split=4,
                                           max_depth=6,
                                           )

In [ ]:
rf_chrono_bis, pi_chrono_bis = train_test_model_rf(X_augmented_2022.drop(["pvotepvoteD", "annee", "pvotepreviouspvoteD"], axis=1), 
                                           X_augmented_2022["pvotepvoteD"],
                                           n_estimators=300,
                                           min_samples_leaf=4,
                                           min_samples_split=4,
                                           max_depth=6,
                                           )

**L'impact de l'ajout de "pvotepreviouspvoteD" est très important du point de vue de la RMSE et du R2 score.**

L'ajout des colonnes delta a nettement amélioré le R2 score par rapport aux prédictions en utilisant que les données d'une année. Ainsi, la variation des indicateurs éco-sociaux influent sur les performances du modèles.
On notera l'importance que prends le vote de l'année précédente sur le vote actuel. 


2000 - 2007 : zone d'étude